In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import time
import random
from datetime import datetime, timedelta
from shapely.geometry import Point
from busSim import BusSim

In [ ]:
DATA_PATH = "./data"
DAY = "monday"
START_TIME = "12:00:00"
ELAPSE_TIME = "00:30:00"
AVG_WALKING_SPEED = 1.4  # 1.4 meters per second
MAX_WALKING_MIN = 10

In [ ]:
def fmin(minutes):
    return f"{minutes // 60}:{minutes % 60}:00"

# doesn't account for time span across 2 days (not implement for across bus schedule: weekday -> weekend)
def every_time(interval, elapse_time):
    start_times = []
    curr = 0
    min_total = 24 * 60 - elapse_time
    while curr <= min_total:
        start_times.append(fmin(curr))
        curr += interval

    return start_times

In [ ]:
start_times = every_time(10, 30)
perf = pd.DataFrame()

ts = time.time()
for t in start_times:
    print(t)
    # t0 = time.time()
    busSim = BusSim(DATA_PATH, DAY, t, ELAPSE_TIME, AVG_WALKING_SPEED, MAX_WALKING_MIN)
    t1 = time.time()
    gdf = busSim.get_gdf(start_point=(43.073691, -89.387407))
    t2 = time.time()
    busSim.get_area(gdf)
    t3 = time.time() 
    # perf.loc[t, "graph-gen"] = t1 - t0
    perf.loc[t, "graph-search"] = t2 - t1
    perf.loc[t, "calc-area"] = t3 - t2
    perf.loc[t, "total"] = t3 - t1
    perf.loc[t, "nodes-num"] = len(busSim.graph.nodes)

te = time.time()
perf

In [ ]:
te-ts

In [ ]:
plt.rcParams["font.size"] = 16

fig, ax = plt.subplots(figsize=(14,8))
perf["total"].plot.line(ax=ax, label="Total", color="k", xlim=10, ylim=0)
perf["graph-search"].plot.line(ax=ax, label="Graph Search", color="0.5")
perf["calc-area"].plot.line(ax=ax, label="Calculate area", color="0.5", linestyle="--")
ax.set_xlabel("Starting time")
ax.set_ylabel("Sim Exec Time (seconds)")
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(14,8))
perf["nodes-num"].plot.line(ax=ax, label="nodes number", color="k", xlim=10, ylim=0)
ax.set_xlabel("Starting time")
ax.set_ylabel("nodes number")
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

perf["nodes-num"].max()

In [ ]:
def gen_locations(num):
    mmt_gtfs_path = os.path.join(DATA_PATH, "mmt_gtfs")
    stops_df = pd.read_csv(os.path.join(
        mmt_gtfs_path, "stops.csv"), sep=",")
    locations = []

    lat_min = stops_df.stop_lat.min()
    lat_max = stops_df.stop_lat.max()
    lon_min = stops_df.stop_lon.min()
    lon_max = stops_df.stop_lon.max()

    for i in range(num):
        lat = random.random() * (lat_max - lat_min) + lat_min
        lon = random.random() * (lon_max - lon_min) + lon_min
        locations.append((lat, lon))

    return locations

In [ ]:
t0 = time.time()
busSim = BusSim(DATA_PATH, DAY, "10:00:00", ELAPSE_TIME, AVG_WALKING_SPEED, MAX_WALKING_MIN)
t1 = time.time()
for location in gen_locations(10):
    print("--------------------------------------------------------------")
    gdf = busSim.get_gdf(start_point=(location[0], location[1]))
    busSim.get_area(gdf)
t2 = time.time()

In [ ]:
from busSim.util import get_perf
from datetime import datetime
import matplotlib

matplotlib.rcParams["font.size"] = 18

In [ ]:
fig, ax = plt.subplots(figsize=(14,8))
perf_df = get_perf("busSim.log")
perf_df.mean().plot.bar(ax = ax)
ax.set_ylabel("execution time (ms)")
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.xticks(rotation=0)